In [2]:
def get_data(endpoint, params):
    URL = "https://v3.football.api-sports.io/"
    headers = {
	'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "fb2140228973d644db847895c454c22b"
    }
    response = requests.get(
        URL+endpoint,
        headers = headers,
        params = params
    )
    if response.status_code == 200:
            
        remaining = response.headers.get("x-ratelimit-requests-remaining")
        data = response.json()
        print(f"requests before reaching limit {remaining}")

    else:
        print(f"Error {response.status_code}, {response.text}")

    return data, remaining                                                                     

In [38]:
def encode_data(data_dict, parent_key = '', sep= '_'):
    encoded = []
    for key, val in data_dict.items():
        new_key = f'{parent_key}{sep}{key}' if parent_key else key
        if isinstance(val, dict):
            encoded.extend(encode_data(val, new_key, sep=sep).items())
        elif isinstance(val, list):
            if val:
                if all(isinstance(i, dict) for i in val):
                    for k, v in enumerate(val):
                        v_key = f'{new_key}{sep}{k}'
                        encoded.extend(encode_data(v, v_key, sep=sep).items())
                else:
                    encoded.append((new_key, val))
            else:
                encoded.append((new_key, []))
        else:
            encoded.append((new_key, val))
    return dict(encoded)

In [50]:
def data_to_sql(table_name, df, db_params, conflict_columns):
    conn = None
    cur = None
    try:
        # Establish the connection
        conn = psycopg2.connect(**db_params)
        cur = conn.cursor()
        
        #insert data into tables
        insert_query = """
            INSERT INTO {} ({})
            VALUES ({})
            ON CONFLICT ({}) DO NOTHING
        """.format(table_name, ','.join(df.columns), ','.join(['%s']*len(df.columns)), ','.join(conflict_columns))
        cur.executemany(insert_query, df.values.tolist())
        print(f'table {table_name} updated')
        
        # Commit the changes
        conn.commit()
        
    
    except Exception as e:
        print(f"Error: {e}")
    
    finally:
        if conn is not None:
            # Close the cursor and connection
            cur.close()
        if cur is not None:
            conn.close()

In [52]:
import requests

endpoint = 'teams/statistics'
params= {'league': 39,
         'team': 40,
         'season': 2023}

df2, remain = get_data(endpoint, params)

requests before reaching limit 98


In [53]:
df2['response']

{'league': {'id': 39,
  'name': 'Premier League',
  'country': 'England',
  'logo': 'https://media.api-sports.io/football/leagues/39.png',
  'flag': 'https://media.api-sports.io/flags/gb.svg',
  'season': 2023},
 'team': {'id': 40,
  'name': 'Liverpool',
  'logo': 'https://media.api-sports.io/football/teams/40.png'},
 'form': 'DWWWWWLDWWDWDWWWDDWWWWLWWWWDWWDLWLDWDW',
 'fixtures': {'played': {'home': 19, 'away': 19, 'total': 38},
  'wins': {'home': 15, 'away': 9, 'total': 24},
  'draws': {'home': 3, 'away': 7, 'total': 10},
  'loses': {'home': 1, 'away': 3, 'total': 4}},
 'goals': {'for': {'total': {'home': 49, 'away': 37, 'total': 86},
   'average': {'home': '2.6', 'away': '1.9', 'total': '2.3'},
   'minute': {'0-15': {'total': 2, 'percentage': '2.44%'},
    '16-30': {'total': 10, 'percentage': '12.20%'},
    '31-45': {'total': 14, 'percentage': '17.07%'},
    '46-60': {'total': 18, 'percentage': '21.95%'},
    '61-75': {'total': 11, 'percentage': '13.41%'},
    '76-90': {'total': 19, 

In [54]:
total_team_stats_data = []
total_team_stats_data.append(encode_data(df2['response']))
total_team_stats_data

[{'league_id': 39,
  'league_name': 'Premier League',
  'league_country': 'England',
  'league_logo': 'https://media.api-sports.io/football/leagues/39.png',
  'league_flag': 'https://media.api-sports.io/flags/gb.svg',
  'league_season': 2023,
  'team_id': 40,
  'team_name': 'Liverpool',
  'team_logo': 'https://media.api-sports.io/football/teams/40.png',
  'form': 'DWWWWWLDWWDWDWWWDDWWWWLWWWWDWWDLWLDWDW',
  'fixtures_played_home': 19,
  'fixtures_played_away': 19,
  'fixtures_played_total': 38,
  'fixtures_wins_home': 15,
  'fixtures_wins_away': 9,
  'fixtures_wins_total': 24,
  'fixtures_draws_home': 3,
  'fixtures_draws_away': 7,
  'fixtures_draws_total': 10,
  'fixtures_loses_home': 1,
  'fixtures_loses_away': 3,
  'fixtures_loses_total': 4,
  'goals_for_total_home': 49,
  'goals_for_total_away': 37,
  'goals_for_total_total': 86,
  'goals_for_average_home': '2.6',
  'goals_for_average_away': '1.9',
  'goals_for_average_total': '2.3',
  'goals_for_minute_0-15_total': 2,
  'goals_for_

In [39]:
total_teams_data = []
total_teams_data.extend(encode_data(row) for row in df['response'])
total_teams_data

[{'team_id': 33,
  'team_name': 'Manchester United',
  'team_code': 'MUN',
  'team_country': 'England',
  'team_founded': 1878,
  'team_national': False,
  'team_logo': 'https://media.api-sports.io/football/teams/33.png',
  'venue_id': 556,
  'venue_name': 'Old Trafford',
  'venue_address': 'Sir Matt Busby Way',
  'venue_city': 'Manchester',
  'venue_capacity': 76212,
  'venue_surface': 'grass',
  'venue_image': 'https://media.api-sports.io/football/venues/556.png'},
 {'team_id': 34,
  'team_name': 'Newcastle',
  'team_code': 'NEW',
  'team_country': 'England',
  'team_founded': 1892,
  'team_national': False,
  'team_logo': 'https://media.api-sports.io/football/teams/34.png',
  'venue_id': 562,
  'venue_name': "St. James' Park",
  'venue_address': 'St. James&apos; Street',
  'venue_city': 'Newcastle upon Tyne',
  'venue_capacity': 52758,
  'venue_surface': 'grass',
  'venue_image': 'https://media.api-sports.io/football/venues/562.png'},
 {'team_id': 35,
  'team_name': 'Bournemouth',
 

In [55]:
import pandas as pd
team_stats_df = pd.DataFrame(total_team_stats_data)

In [58]:
team_stats_df = team_stats_df.drop(columns = {
    'league_name', 
    'league_country', 
    'league_logo', 
    'league_flag', 
    'team_name', 
    'team_logo'})
team_stats_df.head()

,league_id,league_season,team_id,form,fixtures_played_home,fixtures_played_away,fixtures_played_total,fixtures_wins_home,fixtures_wins_away,fixtures_wins_total,...,cards_red_46-60_total,cards_red_46-60_percentage,cards_red_61-75_total,cards_red_61-75_percentage,cards_red_76-90_total,cards_red_76-90_percentage,cards_red_91-105_total,cards_red_91-105_percentage,cards_red_106-120_total,cards_red_106-120_percentage
0,39,2023,40,DWWWWWLDWWDWDWWWDDWWWWLWWWWDWWDLWLDWDW,19,19,38,15,9,24,...,1,20.00%,1,20.00%,1,20.00%,None,None,None,None


In [59]:
a = team_stats_df.columns

In [62]:
for el in a:
    print(el)

league_id
league_season
team_id
form
fixtures_played_home
fixtures_played_away
fixtures_played_total
fixtures_wins_home
fixtures_wins_away
fixtures_wins_total
fixtures_draws_home
fixtures_draws_away
fixtures_draws_total
fixtures_loses_home
fixtures_loses_away
fixtures_loses_total
goals_for_total_home
goals_for_total_away
goals_for_total_total
goals_for_average_home
goals_for_average_away
goals_for_average_total
goals_for_minute_0-15_total
goals_for_minute_0-15_percentage
goals_for_minute_16-30_total
goals_for_minute_16-30_percentage
goals_for_minute_31-45_total
goals_for_minute_31-45_percentage
goals_for_minute_46-60_total
goals_for_minute_46-60_percentage
goals_for_minute_61-75_total
goals_for_minute_61-75_percentage
goals_for_minute_76-90_total
goals_for_minute_76-90_percentage
goals_for_minute_91-105_total
goals_for_minute_91-105_percentage
goals_for_minute_106-120_total
goals_for_minute_106-120_percentage
goals_against_total_home
goals_against_total_away
goals_against_total_total
g

In [46]:
teams_df = teams_df[['team_id', 'team_name', 'team_country', 'team_logo', 'team_national', 'venue_capacity', 'venue_surface']].rename(columns = {'team_national': 'national'})
teams_df

,team_id,team_name,team_country,team_logo,national,venue_capacity,venue_surface
0,33,Manchester United,England,https://media.api-sports.io/football/teams/33.png,False,76212,grass
1,34,Newcastle,England,https://media.api-sports.io/football/teams/34.png,False,52758,grass
2,35,Bournemouth,England,https://media.api-sports.io/football/teams/35.png,False,12000,grass
3,36,Fulham,England,https://media.api-sports.io/football/teams/36.png,False,25700,grass
4,39,Wolves,England,https://media.api-sports.io/football/teams/39.png,False,34624,grass
5,40,Liverpool,England,https://media.api-sports.io/football/teams/40.png,False,55212,grass
6,42,Arsenal,England,https://media.api-sports.io/football/teams/42.png,False,60383,grass
7,44,Burnley,England,https://media.api-sports.io/football/teams/44.png,False,22546,grass
8,45,Everton,England,https://media.api-sports.io/football/teams/45.png,False,40569,grass
9,47,Tottenham,England,https://media.api-sports.io/football/teams/47.png,False,62850,grass


In [51]:
import psycopg2

db_params = {
    'host': 'localhost',
    'database': 'preds',
    'user': 'postgres',
    'password': 'pass',
    'port': '5432'
}

data_to_sql('teams', teams_df, db_params)

table teams updated


In [45]:
import pandas as pd
listwa = []
final = []
for i in range(0,10):
    for j in range(0,10):
        d = (j, j+i)
        listwa.append(d)
        tmp = dict(listwa)
    final.append(tmp)
final


[{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9},
 {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10},
 {0: 2, 1: 3, 2: 4, 3: 5, 4: 6, 5: 7, 6: 8, 7: 9, 8: 10, 9: 11},
 {0: 3, 1: 4, 2: 5, 3: 6, 4: 7, 5: 8, 6: 9, 7: 10, 8: 11, 9: 12},
 {0: 4, 1: 5, 2: 6, 3: 7, 4: 8, 5: 9, 6: 10, 7: 11, 8: 12, 9: 13},
 {0: 5, 1: 6, 2: 7, 3: 8, 4: 9, 5: 10, 6: 11, 7: 12, 8: 13, 9: 14},
 {0: 6, 1: 7, 2: 8, 3: 9, 4: 10, 5: 11, 6: 12, 7: 13, 8: 14, 9: 15},
 {0: 7, 1: 8, 2: 9, 3: 10, 4: 11, 5: 12, 6: 13, 7: 14, 8: 15, 9: 16},
 {0: 8, 1: 9, 2: 10, 3: 11, 4: 12, 5: 13, 6: 14, 7: 15, 8: 16, 9: 17},
 {0: 9, 1: 10, 2: 11, 3: 12, 4: 13, 5: 14, 6: 15, 7: 16, 8: 17, 9: 18}]

In [47]:
res = pd.DataFrame(final)
res

,0,1,2,3,4,5,6,7,8,9
0,0,1,2,3,4,5,6,7,8,9
1,1,2,3,4,5,6,7,8,9,10
2,2,3,4,5,6,7,8,9,10,11
3,3,4,5,6,7,8,9,10,11,12
4,4,5,6,7,8,9,10,11,12,13
5,5,6,7,8,9,10,11,12,13,14
6,6,7,8,9,10,11,12,13,14,15
7,7,8,9,10,11,12,13,14,15,16
8,8,9,10,11,12,13,14,15,16,17
9,9,10,11,12,13,14,15,16,17,18
